# Dog Breed Classification using Pretrained Inception Model

The following code classifies dog breeds using the pretrained inception model

Based on code from Aurelien Geron https://github.com/ageron/handson-ml/blob/master/13_convolutional_neural_networks.ipynb

Dataset: https://www.kaggle.com/c/dog-breed-identification/data

I am not able to run this code on my current machine, so I do not have any performance metrics/

Read in the file paths, and add class labels

In [73]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import flatten

labels = pd.read_csv("./datasets/labels.csv")

TRAIN_PATH="./datasets/train"
TEST_PATH="./datasets/test"
paths_and_classes = []
test_paths = []

dog_class = labels["breed"].unique()
dog_class_ids = {dog_class: index for index, dog_class in enumerate(dog_class)}

for filepath in os.listdir(TRAIN_PATH):
    _id = labels.loc[labels["id"]==filepath.split(".")[0]]
    paths_and_classes.append((os.path.join(TRAIN_PATH,filepath), dog_class_ids[_id.iloc[0]["breed"]]))

for filepath in os.listdir(TEST_PATH):
    test_paths.append(os.path.join(TEST_PATH,filepath))

In [74]:
len(test_paths)

10357

Download incpetion model, A. Geron's method

In [2]:
import sys
import tarfile
from six.moves import urllib

TF_MODELS_URL = "http://download.tensorflow.org/models"
INCEPTION_V3_URL = TF_MODELS_URL + "/inception_v3_2016_08_28.tar.gz"
INCEPTION_PATH = os.path.join("datasets", "inception")
INCEPTION_V3_CHECKPOINT_PATH = os.path.join(INCEPTION_PATH, "inception_v3.ckpt")

def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()

def fetch_pretrained_inception_v3(url=INCEPTION_V3_URL, path=INCEPTION_PATH):
    if os.path.exists(INCEPTION_V3_CHECKPOINT_PATH):
        return
    os.makedirs(path, exist_ok=True)
    tgz_path = os.path.join(path, "inception_v3.tgz")
    urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
    inception_tgz = tarfile.open(tgz_path)
    inception_tgz.extractall(path=path)
    inception_tgz.close()
    os.remove(tgz_path)

Split the data into train and validation sets

In [2]:
split_ratio = .9
num_train = int(len(paths_and_classes)*split_ratio)
np.random.shuffle(paths_and_classes)
train = paths_and_classes[:num_train]
val = paths_and_classes[num_train:]

Crop and resize the image. Method from A. Geron. 
Prepare batch preps the image and labels.

In [3]:
from skimage.transform import resize
import matplotlib.image as mpimg
channels = 3
def prepare_image(image, target_width = 299, target_height = 299, max_zoom = 0.2):
    # First, let's find the largest bounding box with the target size ratio that fits within the image
    height = image.shape[0]
    width = image.shape[1]
    image_ratio = width / height
    target_image_ratio = target_width / target_height
    crop_vertically = image_ratio < target_image_ratio
    crop_width = width if crop_vertically else int(height * target_image_ratio)
    crop_height = int(width / target_image_ratio) if crop_vertically else height
        
    resize_factor = np.random.rand() * max_zoom + 1.0
    crop_width = int(crop_width / resize_factor)
    crop_height = int(crop_height / resize_factor)
    
    x0 = np.random.randint(0, width - crop_width)
    y0 = np.random.randint(0, height - crop_height)
    x1 = x0 + crop_width
    y1 = y0 + crop_height
    
    image = image[y0:y1, x0:x1]
    if np.random.rand() < 0.5:
        image = np.fliplr(image)
    image = resize(image, (target_width, target_height))
    return image

def prepare_batch(index, batch_size,data):
    batch_paths_and_classes = data[index:(index+batch_size)]
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_batch = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    X_batch = np.stack(prepared_images)
    y_batch = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_batch, y_batch

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [75]:
def prepare_test_batch(index, batch_size,data):
    batch_paths = data[index:(index+batch_size)]
    images = [mpimg.imread(path)[:, :, :channels] for path in batch_paths]
    prepared_images = [prepare_image(image) for image in images]
    X_batch = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    X_batch = np.stack(prepared_images)
    return X_batch


Import the inception model and replace the top layer with the number of classes (120). Run standard train and optimization steps. 

In [78]:
X_batch = prepare_test_batch(0,10,test_paths)
X_batch.shape

c:\users\zdunkerton\appdata\local\programs\python\python36\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(10, 299, 299, 3)

In [7]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim
INCEPTION_PATH = os.path.join("datasets", "inception")
INCEPTION_V3_CHECKPOINT_PATH = os.path.join(INCEPTION_PATH, "inception_v3.ckpt")
height = 299
width = 299
channels = 3
reset_graph()
X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()
prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])
extra_layer = tf.layers.dense(prelogits, 750)
extra_layer_2 = tf.layers.dense(extra_layer, 400)
extra_layer_3 = tf.layers.dense(extra_layer_2, 250)
n_outputs = 120

with tf.name_scope("new_output_layer"):
    dog_logits = tf.layers.dense(extra_layer_3, n_outputs, name="dog_logits")
    Y_proba = tf.nn.softmax(dog_logits, name="Y_proba")
    
y = tf.placeholder(tf.int32, shape=[None])

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=dog_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    dog_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="dog_logits")
    training_op = optimizer.minimize(loss, var_list=dog_vars)

with tf.name_scope("eval"):
    #correct = tf.nn.in_top_k(dog_logits, y, 1)
    #accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy = tf.nn.softmax(dog_logits, name="Y_proba")

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver() 

In [5]:
from time import time
import sys
batch_size = 40
n_epochs = 10
with tf.Session() as sess:
    init.run()
    inception_saver.restore(sess, INCEPTION_V3_CHECKPOINT_PATH)
    for epoch in range(n_epochs):
        for iteration in range(len(train) // batch_size):
            t0 = time()
            X_batch, y_batch = prepare_batch(batch_size*iteration, batch_size, train)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            t1 = time()
            time_remaining = ((len(train) // batch_size)-iteration)*(t1-t0)
            time_remaining = time_remaining/60
            sys.stdout.write("\rIterations Remaining: {0:.2f}, Time {1:.2f}s, Est. Time Remainig: {2:.2f} min".format(((len(train) // batch_size)-iteration),((t1-t0)),time_remaining))
            sys.stdout.flush()
        scores = []
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print("  Train accuracy:", acc_train)
        save_path = saver.save(sess, "./dog_breed_model")

INFO:tensorflow:Restoring parameters from datasets\inception\inception_v3.ckpt


c:\users\zach dunkerton\appdata\local\programs\python\python36\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Iterations Remaining: 1.00, Time 15.61s, Est. Time Remainig: 0.26 minnnn  Train accuracy: 0.775
Iterations Remaining: 1.00, Time 15.00s, Est. Time Remainig: 0.25 minnnn  Train accuracy: 0.775
Iterations Remaining: 1.00, Time 14.80s, Est. Time Remainig: 0.25 minnnn  Train accuracy: 0.9
Iterations Remaining: 1.00, Time 14.78s, Est. Time Remainig: 0.25 minnnn  Train accuracy: 0.825
Iterations Remaining: 1.00, Time 14.91s, Est. Time Remainig: 0.25 minnnn  Train accuracy: 0.925
Iterations Remaining: 1.00, Time 14.83s, Est. Time Remainig: 0.25 minnnn  Train accuracy: 0.9
Iterations Remaining: 1.00, Time 14.75s, Est. Time Remainig: 0.25 minnnn  Train accuracy: 0.9
Iterations Remaining: 1.00, Time 14.73s, Est. Time Remainig: 0.25 minnnn  Train accuracy: 0.9
Iterations Remaining: 1.00, Time 14.85s, Est. Time Remainig: 0.25 minnnn  Train accuracy: 0.95
Iterations Remaining: 1.00, Time 14.73s, Est. Time Remainig: 0.25 minnnn  Train accuracy: 0.95


In [ ]:
test_batch_size=1
probabilities = []
from time import time
import sys

with tf.Session() as sess:
    saver.restore(sess, "./model/dog_breed_model")
    for iteration in range(len(test_paths) // test_batch_size):
        t0 = time()
        X_test_batch = prepare_test_batch(test_batch_size*iteration, test_batch_size, test_paths)
        probabilities.append(accuracy.eval(feed_dict={X: X_test_batch}))
        t1 = time()
        time_remaining = ((len(test_paths) // test_batch_size)-iteration)*(t1-t0)
        time_remaining = time_remaining/60
        sys.stdout.write("\rIterations Remaining: {0:.2f}, Time {1:.2f}s, Est. Time Remainig: {2:.2f} min".format(((len(test_paths) // test_batch_size)-iteration),((t1-t0)),time_remaining))            
        sys.stdout.flush()

INFO:tensorflow:Restoring parameters from ./model/dog_breed_model


c:\users\zdunkerton\appdata\local\programs\python\python36\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Iterations Remaining: 5721.00, Time 0.35s, Est. Time Remainig: 33.75 minnn

In [81]:
arr = np.array(acc_vals)
arr.shape
#arr = np.reshape(arr, (1023,120))
#print(arr.shape)
#df = pd.DataFrame(arr)

(39, 1, 120)

In [85]:
range(len(test_paths) // test_batch_size)

range(0, 10357)

In [54]:
dog_class = labels["breed"].unique()
df.columns = dog_class
df.to_csv("./proba.csv")

In [61]:
df = df.reindex_axis(sorted(df.columns), axis=1)

In [63]:
df.to_csv("./proba.csv")

In [68]:
val[13]

('./datasets/train\\a77619f67680f34bb77a80d67e6efa70.jpg', 40)

In [65]:
dog_class_ids

{'affenpinscher': 24,
 'afghan_hound': 74,
 'african_hunting_dog': 13,
 'airedale': 63,
 'american_staffordshire_terrier': 68,
 'appenzeller': 46,
 'australian_terrier': 42,
 'basenji': 7,
 'basset': 41,
 'beagle': 95,
 'bedlington_terrier': 5,
 'bernese_mountain_dog': 67,
 'black-and-tan_coonhound': 22,
 'blenheim_spaniel': 110,
 'bloodhound': 47,
 'bluetick': 3,
 'border_collie': 53,
 'border_terrier': 33,
 'borzoi': 6,
 'boston_bull': 0,
 'bouvier_des_flandres': 115,
 'boxer': 17,
 'brabancon_griffon': 80,
 'briard': 91,
 'brittany_spaniel': 50,
 'bull_mastiff': 66,
 'cairn': 23,
 'cardigan': 70,
 'chesapeake_bay_retriever': 92,
 'chihuahua': 58,
 'chow': 82,
 'clumber': 72,
 'cocker_spaniel': 118,
 'collie': 55,
 'curly-coated_retriever': 106,
 'dandie_dinmont': 93,
 'dhole': 31,
 'dingo': 1,
 'doberman': 18,
 'english_foxhound': 87,
 'english_setter': 27,
 'english_springer': 117,
 'entlebucher': 54,
 'eskimo_dog': 78,
 'flat-coated_retriever': 83,
 'french_bulldog': 114,
 'german